In [5]:
import pandas as pd
from tqdm import tqdm
import os
import nltk
from nltk.corpus import stopwords

In [7]:
brexit_file = pd.read_csv('./brexit_full_updated4.csv')

politicians = []

count_zero = 0

politicians_party ={}
politicians_party2 ={}

for index, row in brexit_file.iterrows():
    full_name = row['full name']
    party = row['party']
    party2 = row['Party abbreviation']
    last_name = full_name.split(" ")[-1]
    first_name = full_name.split(" ")[0]
    short_name = last_name.lower() + '.' + first_name.lower()

    if party == 0:
        count_zero += 1
    if party in [0, 1]:
        politicians.append(short_name)
        politicians_party[short_name] = party
    politicians_party2[short_name] = party2

print(len(politicians))
politicians = list(set(politicians))
print(len(politicians))

594
594


In [8]:
politicians_party

{'howarth.gerald': 1.0,
 'brady.graham': 1.0,
 'mills.nigel': 1.0,
 'herbert.nick': 0.0,
 'piero.gloria': 0.0,
 'green.damian': 0.0,
 'rayner.angela': 0.0,
 'lidington.david': 0.0,
 'prentis.victoria': 0.0,
 'hodge.margaret': 0.0,
 'jarvis.dan': 0.0,
 'dugher.michael': 0.0,
 'woodcock.john': 0.0,
 'baron.john': 1.0,
 'miller.maria': 0.0,
 'howlett.ben': 0.0,
 'cox.jo': 0.0,
 'ellison.jane': 0.0,
 'grieve.dominic': 0.0,
 'fuller.richard': 1.0,
 'coyle.neil': 0.0,
 'trevelyan.anne-marie': 1.0,
 'ali.rushanara': 0.0,
 'stuart.graham': 0.0,
 'field.frank': 1.0,
 'stuart.gisela': 1.0,
 'dromey.jack': 0.0,
 'godsiff.roger': 1.0,
 'byrne.liam': 0.0,
 'mahmood.shabana': 0.0,
 'burden.richard': 0.0,
 'mahmood.khalid': 1.0,
 'mccabe.steve': 0.0,
 'phillips.jess': 0.0,
 'goodman.helen': 0.0,
 'hollern.kate': 0.0,
 'stringer.graham': 1.0,
 'maynard.paul': 0.0,
 'marsden.gordon': 0.0,
 'anderson.dave': 0.0,
 'campbell.ronnie': 1.0,
 'gibb.nick': 0.0,
 'crausby.david': 0.0,
 'qureshi.yasmin': 0.0,
 

In [9]:
res_df = pd.DataFrame(politicians, columns=['politician last name'])
res_df

,politician last name
0,mowat.david
1,burden.richard
2,zeichner.daniel
3,campbell.gregory
4,lewis.brandon
...,...
589,meacher.michael
590,walker.robin
591,mccartney.jason
592,davies.david


In [10]:
res_df.to_csv('politicians_names.csv')

In [11]:
count_zero

449

In [12]:
prefix_map ={
    54: 'BBC+News',
    106: 'News',
    107: 'News',
    175: 'News',
    279: ''
}

In [14]:
bbc_ids = [54, 106, 107, 175, 279]
#, 106, 107, 175, 279
partition = 1

# BBC

In [29]:
stopW = stopwords.words('english')

def contains_european(x):
    y = x.lower()
    words = y.split()
    flag = 'e.u' in words or 'eu' in words or 'europeanunion' in words
    words = [word for word in words if word not in stopW]
    bigrams = list(nltk.bigrams(words))
    bigrams = [bigram[0] + '.' + bigram[1] for bigram in bigrams]
    
    flag = flag or 'european.union' in bigrams or 'europe.union' in bigrams
    
    return 'EU' if flag else 'non_EU'

# programs = ['BBC News at One', 'BBC News at Six', 'BBC News at Ten']
def get_df(bbc_id):
    if bbc_id == 54:
        path = '../data/partition_predictions/window_topic_prediction_with_short_sentences_merged_{}_{}.csv'
        
        path1 = path.format(bbc_id, 2015)
        path2 = path.format(bbc_id, 2016)
        print(path1)
        print(path2)

        df1 = pd.read_csv(path1)
        df1['date'] = pd.to_datetime(df1['date'])
        df1['month'] = df1['date'].apply(lambda x: x.month)
        df1 = df1.loc[df1.month >= 6]
        df2 = pd.read_csv(path2)
        df2['date'] = pd.to_datetime(df2['date'])
        df2['month'] = df2['date'].apply(lambda x: x.month)
        df2 = df2.loc[df2.month <= 6]
        df = pd.concat([df1, df2])
        df = df.drop(['Unnamed: 0', 'month'], axis=1)
        rows = []

        last_topic = None
        last_partition = None
        for index, row in df.iterrows():
            partition_id = row['partition_id']
            dt = row['date']
            source = row['source']
            transcript = row['transcript']
            t = row['type']
            topic = row['topic']


            if last_partition == partition_id and last_topic == topic:
                rows[-1][3] += row['transcript']
            else:
                last_topic = topic
                last_partition = partition_id
                rows.append([partition_id, dt, source, transcript, t, topic])
        df = pd.DataFrame(rows, columns=df.columns)
#         df = df.loc[df['source'].isin(programs)]
        display(df.head())
    else:
        path = '../data/partitions/bert_partitions_{}_{}.csv'
    
        path1 = path.format(bbc_id, 2015)
        path2 = path.format(bbc_id, 2016)
        print(path1)
        print(path2)

        df1 = pd.read_csv(path1)
        if 'Unnamed: 0' in df1.columns:
            df1 = df1.drop(['Unnamed: 0'], axis=1)
        df1.columns = ['Source','date', 'Program Name', 'Time', 'Duration', 'transcript', 'partitioned_transcript']
        df1['date'] = pd.to_datetime(df1['date'])
        df1['month'] = df1['date'].apply(lambda x: x.month)
        df1 = df1.loc[df1.month >= 6]
        df2 = pd.read_csv(path2)
        if 'Unnamed: 0' in df2.columns:
            df2 = df2.drop(['Unnamed: 0'], axis=1)
        df2.columns = ['Source','date', 'Program Name', 'Time', 'Duration', 'transcript', 'partitioned_transcript']
        df2['date'] = pd.to_datetime(df2['date'])
        df2['month'] = df2['date'].apply(lambda x: x.month)
        df2 = df2.loc[df2.month <= 6]
        df = pd.concat([df1, df2])
        df = df.drop(['month'], axis=1)
        display(df.head())
    
        partitions = []
        for index, row in df.iterrows():
            partition_string = row['partitioned_transcript']
            partition_date = row['date']
            all_partitions = str(partition_string).split('\n---------------------\n')
            for part in all_partitions:
                partitions.append((index, partition_date, part))
        df = pd.DataFrame(partitions, columns=['partition_id', 'date', 'transcript'])
        
    df['transcript'] = df['transcript'].apply(lambda x: x.lower())
    df['month'] = df['date'].apply(lambda x: x.month)
    df['year'] = df['date'].apply(lambda x: x.year)
    df['day'] = df['date'].apply(lambda x: x.day)
#     df = df.loc[~((df['year'] == 2016) & (df['month'] == 6) & (df['day'] > 22))]
    df['eu'] = df['transcript'].apply(lambda x: contains_european(x))
    df = df.loc[df['eu'] == 'non_EU']
    
    return df

In [30]:
df = get_df(54)

../data/partition_predictions/window_topic_prediction_with_short_sentences_merged_54_2015.csv
../data/partition_predictions/window_topic_prediction_with_short_sentences_merged_54_2016.csv


KeyboardInterrupt: 

In [ ]:
df

# Count Functions

In [ ]:
# count of times last names appears when first name also appears
def count1(text, last_name, first_name):
    words = text.split(" ")
    
    if first_name in words and last_name in words:
        return words.count(last_name)
    else:
        return 0

# count of articles where first name and last name both appears
def count2(text, last_name, first_name):
    words = text.split(" ")
    
    if first_name in words and last_name in words:
        return 1
    else:
        return 0

# count when last name and first name appears together
def count3(text, last_name, first_name):
    words = text.split(" ")
    
    n_words = len(words)
    
    c = 0
    for i in range(n_words-1):
        if words[i] == first_name and words[i+1] == last_name:
            c += 1

    return c

# count when last name and first name appears together
def count3_new(text, last_name, first_name):
    return text.count(first_name + ' ' + last_name) + text.count(first_name + last_name)

# count when last name and first name appears together
def count4(text, last_name, first_name):
    words = text.split(" ")
    
    n_words = len(words)
    
    for i in range(n_words-1):
        if words[i] == first_name and words[i+1] == last_name:
            return 1

    return 0

def count4_new(text, last_name, first_name):
    patt1 = first_name + ' ' + last_name
    patt2 = first_name + last_name
    
    if patt1 in text or patt2 in text:
        return 1
    
    return 0

# Last Name appears

In [ ]:
for bbc_id in bbc_ids:
    df = get_df(bbc_id)
    
    if partition:
        for p in tqdm(politicians):
            last_name, first_name = p.split('.')
            df[p] = df['transcript'].apply(lambda x: count2(x, last_name, first_name))
    else:
        for p in tqdm(politicians):
            df[p] = df['Transcript'].apply(lambda x: x.count(p))
            
    overall_count_df = None
    
    display(df)
    
    for p in politicians:
        count_df = df[['month', 'year', p]]
        count_df = count_df.groupby(['month', 'year'], as_index=False).sum()

        if overall_count_df is None:
            overall_count_df = count_df
        else:
            overall_count_df[p] = count_df[p]
    overall_count_df = overall_count_df.sort_values(by=['year', 'month']).reset_index(drop=True)
    overall_count_df
                      
    overall_count_df.to_csv('politicians_last_name_count_bbc_{}.csv'.format(bbc_id), index=False)
    rows = []

    for index, row in overall_count_df.iterrows():
        party_count = [0, 0]
        for p in politicians:
            if politicians_party[p] == 0:
                party_count[0] += row[p]
            else:
                party_count[1] += row[p]
        rows.append([row['month'], row['year'], party_count[0], party_count[1]])

    res_df = pd.DataFrame(rows, columns=['month', 'year', 'party=0', 'party=1'])
    res_df = res_df.sort_values(by=['year', 'month']).reset_index(drop=True)
    res_df
    res_df.to_csv('politicians_last_name_count_bbc_{}_by_party.csv'.format(bbc_id), index=False)

# Last name and First Name appears together

In [31]:
for bbc_id in tqdm(bbc_ids):
    print(bbc_id)
    df = get_df(bbc_id)
    
    if partition:
        for p in politicians:
            last_name, first_name = p.split('.')
            df[p] = df['transcript'].apply(lambda x: count3_new(x, last_name, first_name))
    else:
        for p in tqdm(politicians):
            df[p] = df['Transcript'].apply(lambda x: x.count(p))
    
    display(df)
    overall_count_df = None

    for p in politicians:
        count_df = df[['month', 'year', p]]
        count_df = count_df.groupby(['month', 'year'], as_index=False).sum()

        if overall_count_df is None:
            overall_count_df = count_df
        else:
            overall_count_df[p] = count_df[p]
    overall_count_df = overall_count_df.sort_values(by=['year', 'month']).reset_index(drop=True)
    display(overall_count_df)

    overall_count_df.to_csv('politicians_last_name_count_bbc_{}.csv'.format(bbc_id), index=False)

    rows = []

    for index, row in overall_count_df.iterrows():
        party_count = [0, 0, 0, 0]
        for p in politicians:
            if politicians_party[p] == 0:
                party_count[0] += row[p]
            else:
                party_count[1] += row[p]
            
            if politicians_party2[p] == 'Con':
                party_count[2] += row[p]
            else:
                party_count[3] += row[p]
        rows.append([row['month'], row['year'], party_count[0], party_count[1], party_count[2], party_count[3]])

    res_df = pd.DataFrame(rows, columns=['month', 'year', 'party=0', 'party=1', 'conservative', 'labor'])
    res_df = res_df.sort_values(by=['year', 'month']).reset_index(drop=True)
    display(res_df)

    res_df.to_csv('politicians_last_name_count_bbc_{}_by_party.csv'.format(bbc_id), index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

54
../data/partition_predictions/window_topic_prediction_with_short_sentences_merged_54_2015.csv
../data/partition_predictions/window_topic_prediction_with_short_sentences_merged_54_2016.csv


,partition_id,date,source,transcript,type,topic
0,943,2015-06-06,BBC News,Good afternoon. David Cameron will urge world...,p,Transport
1,943,2015-06-06,BBC News,Four rollercoasters at three theme parks in t...,p,"Culture, media and sport"
2,943,2015-06-06,BBC News,Four people remain in hospital after the cras...,p,Transport
3,943,2015-06-06,BBC News,Pope Francis has arrived in the Bosnian capit...,p,International affairs
4,943,2015-06-06,BBC News,Chinese authorities have said they now believ...,p,Transport


,partition_id,date,source,transcript,type,topic,month,year,day,eu,...,garnier.mark,brown.alan,shelbrooke.alec,coffey.ann,lewis.julian,meacher.michael,walker.robin,mccartney.jason,davies.david,rees.christina
0,943,2015-06-06,BBC News,good afternoon. david cameron will urge world...,p,Transport,6,2015,6,non_EU,...,0,0,0,0,0,0,0,0,0,0
1,943,2015-06-06,BBC News,four rollercoasters at three theme parks in t...,p,"Culture, media and sport",6,2015,6,non_EU,...,0,0,0,0,0,0,0,0,0,0
2,943,2015-06-06,BBC News,four people remain in hospital after the cras...,p,Transport,6,2015,6,non_EU,...,0,0,0,0,0,0,0,0,0,0
3,943,2015-06-06,BBC News,pope francis has arrived in the bosnian capit...,p,International affairs,6,2015,6,non_EU,...,0,0,0,0,0,0,0,0,0,0
4,943,2015-06-06,BBC News,chinese authorities have said they now believ...,p,Transport,6,2015,6,non_EU,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54021,1284,2016-05-31,BBC News at Ten,"our north america editor, jon sopel, is in wa...",p,International affairs,5,2016,31,non_EU,...,0,0,0,0,0,0,0,0,0,0
54022,1284,2016-05-31,BBC News at Ten,"well, england and wales have named their squa...",p,"Culture, media and sport",5,2016,31,non_EU,...,0,0,0,0,0,0,0,0,0,0
54025,1284,2016-05-31,BBC News at Ten,tonight we reveal startling new figures showi...,p,Housing and planning,5,2016,31,non_EU,...,0,0,0,0,0,0,0,0,0,0
54026,1284,2016-05-31,BBC News at Ten,"along with more affordable homes, sadiq khan ...",p,"Parliament, government and politics",5,2016,31,non_EU,...,0,0,0,0,0,0,0,0,0,0


,month,year,mowat.david,burden.richard,zeichner.daniel,campbell.gregory,lewis.brandon,hunt.jeremy,burgon.richard,robinson.gavin,...,garnier.mark,brown.alan,shelbrooke.alec,coffey.ann,lewis.julian,meacher.michael,walker.robin,mccartney.jason,davies.david,rees.christina
0,6,2015,0,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,2015,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,2,0
2,8,2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,2015,0,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
4,10,2015,0,0,0,0,0,3,0,0,...,0,0,0,0,0,1,0,0,0,0
5,11,2015,0,0,0,0,0,21,0,0,...,0,0,0,0,0,0,0,0,0,0
6,12,2015,0,0,0,0,0,8,0,0,...,0,0,0,0,0,1,0,0,0,0
7,1,2016,0,0,0,0,1,12,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,2016,0,0,0,0,0,62,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,2016,0,0,0,0,0,9,0,0,...,0,0,0,0,0,0,0,0,0,0


...

,Source,date,Program Name,Time,Duration,transcript,partitioned_transcript
0,Sky News,2015-10-04,CBS News,05:30,30 mins,~ Subtitles by Ericsson. E-mail: accessibility...,An eight-year old boy and a pensioner die in a...
1,Sky News,2015-10-04,Sunrise,06:00,240 mins,Subtitles by Ericsson. E-mail: accessibility@b...,Breaking news - at least 13 dead in flash floo...
2,Sky News,2015-10-02,Sunrise,06:00,180 mins,I won’t be the last one to find a lump. I won’...,I won’t be the last one to find a lump. I won’...
3,Sky News,2015-10-02,Sky News with Colin Brazier and Jayne Secker,09:00,180 mins,"I need injury compensation, but I don’t know w...","I need injury compensation, but I don’t know w..."
4,Sky News,2015-10-09,Sky World News,04:00,30 mins,Be one with your tyres and the road will be on...,Be one with your tyres and the road will be on...


,partition_id,date,transcript,month,year,day,eu,mowat.david,burden.richard,zeichner.daniel,...,garnier.mark,brown.alan,shelbrooke.alec,coffey.ann,lewis.julian,meacher.michael,walker.robin,mccartney.jason,davies.david,rees.christina
0,0,2015-10-04,an eight-year old boy and a pensioner die in a...,10,2015,4,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2015-10-04,paid leave for grandparents - the chancellor o...,10,2015,4,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2015-10-04,president obama promises a full investigation ...,10,2015,4,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2015-10-04,disappointment for fans as hosts england are d...,10,2015,4,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,2015-10-04,breaking news - at least 13 dead in flash floo...,10,2015,4,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17385,734,2016-05-28,hull beat sheffield wednesday in the champions...,5,2016,28,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17386,734,2016-05-28,daniel ricciardo says his first-ever pole posi...,5,2016,28,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17387,734,2016-05-28,"in rugby union, saracens are the first club in...",5,2016,28,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17388,734,2016-05-28,"still to come, the sunday papers for two night...",5,2016,28,non_EU,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,month,year,mowat.david,burden.richard,zeichner.daniel,campbell.gregory,lewis.brandon,hunt.jeremy,burgon.richard,robinson.gavin,...,garnier.mark,brown.alan,shelbrooke.alec,coffey.ann,lewis.julian,meacher.michael,walker.robin,mccartney.jason,davies.david,rees.christina
0,6,2015,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,2015,0,0,0,0,0,4,0,0,...,0,0,0,0,1,0,0,0,0,0
2,8,2015,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,2015,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,1,0
4,10,2015,0,0,0,0,0,11,0,0,...,0,0,0,0,0,1,0,0,0,0
5,11,2015,0,0,0,0,0,10,0,0,...,0,0,0,0,0,1,0,0,0,0
6,12,2015,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
7,1,2016,0,0,0,0,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2,2016,0,0,0,0,0,10,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,2016,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


,month,year,party=0,party=1,conservative,labor
0,6,2015,135,10,104,41
1,7,2015,398,343,614,127
2,8,2015,122,1,19,104
3,9,2015,406,5,63,348
4,10,2015,303,63,248,118
5,11,2015,305,0,196,109
6,12,2015,273,12,119,166
7,1,2016,324,5,164,165
8,2,2016,136,53,179,10
9,3,2016,30,7,35,2


100%|██████████| 5/5 [03:37<00:00, 43.43s/it]


# News

In [58]:
news_df1 = pd.read_csv('../data/news_predictions/news_2015_predictions.csv')
news_df1 = news_df1.drop(['Unnamed: 0', 'parliament', 'top1_acc', 'top1_topic', 'top2_topic', 'top2_acc', 'top3_topic', 'top3_acc'], axis=1)
news_df1 = news_df1.reset_index(drop=True)
news_df1['transcript'] = news_df1['transcript'].apply(lambda x: str(x).lower())
news_df1 = news_df1.loc[news_df1.month >= 6]

In [59]:
news_df2 = pd.read_csv('../data/news_predictions/news_2016_predictions.csv')
news_df2 = news_df2.drop(['Unnamed: 0', 'parliament', 'top1_acc', 'top1_topic', 'top2_topic', 'top2_acc', 'top3_topic', 'top3_acc'], axis=1)
news_df2 = news_df2.reset_index(drop=True)
news_df2['transcript'] = news_df2['transcript'].apply(lambda x: str(x).lower())
news_df2 = news_df2.loc[news_df2.month <= 6]

In [60]:
news_df = pd.concat([news_df1, news_df2])
news_df = news_df.loc[news_df.source_id.isin([138620, 389195, 397135, 8200])]

In [61]:
news_df['eu'] = news_df['transcript'].apply(lambda x: contains_european(x))
# news_df = news_df.loc[~((news_df['year'] == 2016) & (news_df['month'] == 6) & (news_df['day'] > 22))]
news_df = news_df.loc[news_df['eu'] == 'non_EU']

In [62]:
news_df

,source_id,source,day,month,year,program_name,transcript,eu
631053,138620,Guardian,1,6,2015,Office employees should be on feet for four ho...,research has long linked excessive time spent ...,non_EU
631054,138620,Guardian,1,6,2015,Andy Murray\nsets up French Open quarter-final...,"he won 6-4, 3-6, 6-3, 6-2 in just under three ...",non_EU
631055,138620,Guardian,1,6,2015,'I start to feel it in my knees' - working for...,"the us secretary of state donald rumsfeld, fam...",non_EU
631056,138620,Guardian,1,6,2015,Sepp Blatter's Fifa re-election causes anger b...,blatter secured a fifth four-year term in offi...,non_EU
631057,138620,Guardian,1,6,2015,"Greek people, not Syriza, will achieve a break...",then greece's new representative at the imf wi...,non_EU
...,...,...,...,...,...,...,...,...
970081,389195,Telegraph,30,6,2016,Video shows moment entire house topples over a...,no injuries were reported as the area was evac...,non_EU
970082,389195,Telegraph,30,6,2016,Every children's intensive care unit to be pro...,an investigation into a string of deaths in br...,non_EU
970084,389195,Telegraph,30,6,2016,"Kids discuss HS2, the Queen and the state of E...","in this video, we ask some of this country's y...",non_EU
970085,389195,Telegraph,30,6,2016,This family spent their holiday in Turkey maki...,"""the emotional intensity of their situation wa...",non_EU


In [63]:
news_df.source.unique()

array(['Guardian', 'Independent', 'Mail', 'Telegraph'], dtype=object)

In [64]:
for p in tqdm(politicians):
    last_name, first_name = p.split('.')
    news_df[p] = news_df['transcript'].apply(lambda x: count3_new(x, last_name, first_name))

100%|██████████| 594/594 [40:20<00:00,  4.07s/it]


In [65]:
overall_count_df = None

for p in tqdm(politicians):
    count_df = news_df[['source', 'month', 'year', p]]
    count_df = count_df.groupby(['source', 'month', 'year'], as_index=False).sum()
    
    if overall_count_df is None:
        overall_count_df = count_df
    else:
        overall_count_df[p] = count_df[p]

overall_count_df = overall_count_df.sort_values(by=['year', 'month']).reset_index(drop=True)
overall_count_df

100%|██████████| 594/594 [00:44<00:00, 13.39it/s]


,source,month,year,mowat.david,burden.richard,zeichner.daniel,campbell.gregory,lewis.brandon,hunt.jeremy,burgon.richard,...,garnier.mark,brown.alan,shelbrooke.alec,coffey.ann,lewis.julian,meacher.michael,walker.robin,mccartney.jason,davies.david,rees.christina
0,Guardian,6,2015,0,0,0,0,3,38,0,...,0,0,1,0,3,2,0,4,3,0
1,Independent,6,2015,0,2,0,0,1,34,0,...,4,0,1,0,4,0,0,0,0,0
2,Mail,6,2015,0,1,1,0,1,27,1,...,3,0,2,2,2,2,0,1,2,1
3,Telegraph,6,2015,0,0,0,0,5,16,0,...,0,0,0,1,1,0,0,2,1,0
4,Guardian,7,2015,0,0,1,0,1,83,6,...,3,0,0,0,6,4,0,3,8,0
5,Independent,7,2015,0,1,1,1,0,55,4,...,1,0,0,1,16,2,2,0,2,3
6,Mail,7,2015,0,0,0,3,0,71,0,...,8,0,0,0,5,3,1,1,0,0
7,Telegraph,7,2015,0,0,2,0,1,25,1,...,0,0,0,0,8,3,0,0,0,0
8,Guardian,8,2015,0,0,0,2,5,10,0,...,0,0,0,0,0,7,0,0,7,0
9,Independent,8,2015,0,4,0,0,4,12,7,...,0,0,0,2,3,0,0,0,0,0


In [66]:
overall_count_df.to_csv('politicians_last_name_count_news.csv')

In [67]:
rows = []

for index, row in overall_count_df.iterrows():
    party_count = [0, 0, 0, 0]
    for p in politicians:
        if politicians_party[p] == 0:
            party_count[0] += row[p]
        else:
            party_count[1] += row[p]
        if politicians_party2[p] == 'Con':
            party_count[2] += row[p]
        else:
            party_count[3] += row[p]
            
    rows.append([row['source'], row['month'], row['year'], party_count[0], party_count[1], party_count[2], party_count[3]])
    
res_df = pd.DataFrame(rows, columns=['source', 'month', 'year', 'party=0', 'party=1', 'conservative', 'labor'])
res_df = res_df.sort_values(by=['year', 'month']).reset_index(drop=True)
res_df

,source,month,year,party=0,party=1,conservative,labor
0,Guardian,6,2015,1992,241,1109,1124
1,Independent,6,2015,2138,363,1232,1269
2,Mail,6,2015,2146,358,1227,1277
3,Telegraph,6,2015,1163,183,905,441
4,Guardian,7,2015,2612,380,1604,1388
5,Independent,7,2015,3198,442,1712,1928
6,Mail,7,2015,2993,452,1993,1452
7,Telegraph,7,2015,1674,253,1182,745
8,Guardian,8,2015,1868,166,692,1342
9,Independent,8,2015,2347,206,883,1670


In [68]:
res_df.to_csv('politicians_last_name_count_news_by_party.csv')